## ❓ Pre-module quiz

Why is Naive Bayes "naive"?

A. Because it's the most basic, i.e. "naive" classifier we can build

B. Because it "naively" assumes that the probabilities of features (i.e., in our case, words) are independent of each other

C. Because the guy who invented it tought it was a cool name

D. Because it "naively" assumes that the probabilities of features (i.e., in our case, words) are dependent of each other

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
  <p>The correct answer is B - Naive Bayes assumes that the probability of finding a certain word is independent from the probability of finding another word. So, for example, in the domain of movie reviews, it assumes that the probabilities of finding the words <code>Indiana</code> and <code>Jones</code> are not correlated, even if in practice we know that this is not the case.</p>

</details> 


# Python for Computational Linguists 1.2: Breaking down Naive Bayes

## Introduction

Welcome to module 2.1! In this module we will review Naive Bayes and we will write our own implementation of the algorithm. We will use it to classify some toy examples, and then we will turn to Sentiment Analysis, by training our simple model on a research dataset.

### (⚠️ UPDATE ME!) Recap of last notebook

Module 2.1 introduced lots of new concepts and libraries - pre-processing, feature vectors, `numpy`, and so on. In fact, we have seen how to import data from a research dataset, how to clean them by removing punctuation and stop-words, how to use `numpy` to prepare test and training data for a model, and how to use `scikit-learn` to train, test and evaluate a simple Naive Bayes classifier for sentiment analysis.

However, we didn't look 'behind the scenes' at how Naive Bayes actually works. This module will guide you through your first hand-written machine learning model, by showing you how to write the maths for Naive Bayes yourself. 

### Naive Bayes refresher

> Note: this section borrows heavily from the Naive Bayes chapter of the lecture notes. Please refer to them or to the J&M for more details.

Naive Bayes is a simple classifier based on two assumptions:
- The **bag-of-words** assumption: word ordering doesn't matter. We represent each document in our dataset as a list of pairs $(word_i,frequency_i)$.
- The **conditional independence** assumption: the probability of one word appearing in a sentence is by no means correlated to the occurrence of another word.

These two assumptions heavily simplify our model, since they completely disregard grammar and and any domain knowledge (e.g. the `Indiana Jones` example in the pre-module quiz); but as we know now, they allow us to build a surprisingly efficient classifier.

Let's quickly go through the maths of Naive Bayes. Remember that given a document $d$ and a set of classes $C$, we need to assign the document to the class $\hat{c} \in C$ which has the maximum *a posteriori probability*, i.e. where $\hat{c}$ is defined as follows:

$$ \hat{c} = \text{argmax}_{c \in C} P(c \mid d)$$

How do we calculate $\hat{c}$ efficiently? Well, the Bayes Rule tells us that

$$ P(c \mid d) = \frac{P(c) \ P(d \mid c)}{P(d)} $$

Allowing us to rewrite 

$$\begin{align} 
\hat{c} &= \text{argmax}_{c \in C} P(c \mid d) \\
        &= \text{argmax}_{c \in C} \frac{P(c) \ P(d \mid c)}{P(d)}
\end{align}$$

However, the probability $d$ is constant for each class $c$, hence we can remove it, leaving only:

$$
\hat{c} = \text{argmax}_{c \in C} 
    \underbrace{P(c)}_\text{prior}
    \underbrace{P(d \mid c)}_\text{likelihood}
$$

Where the $prior$ is the **prior probability** of the class $c$ and the $likelihood$ is the probability of finding $d$ given the class $c$.

Using words as features, we can represent $d$ as a list of words $w_1, \dots , w_n$, hence 

$$
\hat{c} = \text{argmax}_{c \in C} 
    \underbrace{P(c)}_\text{prior}
    \underbrace{P(w_1, \dots , w_n \mid c)}_\text{likelihood} 
$$

However, $P(w_1, \dots , w_n \mid c)$ may be prohibitively hard to calculate, since we would need to estimate the probability of every possible combination of words. Here, the **conditional independence** assumption comes to the rescue, assuming the probability of the words (i.e. features) are independent, allowing us to finally rewrite

$$
\begin{align}
\hat{c} &= \text{argmax}_{c \in C} 
       \underbrace{P(c)}_\text{prior}
       \underbrace{P(w_1 \mid c) \times \dots \times P(w_n \mid c)}_\text{likelihood} 
       \\
       &= \text{argmax}_{c \in C} 
       \underbrace{P(c)}_\text{prior}
       \underbrace{\prod_{w \in d}{P(w \mid c)}}_\text{likelihood} 
\end{align}
$$

What does all of this mean in practice? Well, that if we have a document $d$, all we need to know to classify it is:
- The *priors*, i.e. the probability of document $d$ to belong to each class $c$
- The *likelihoods*, i.e. the probabilities for each word $w_i$ of the document to belong to each class $c$.

## Naive Bayes: a simple implementation

Let's begin with a simple example from the Post Lecture exercises (taken from J&M-3, exercise 4.2). Given the following short movie reviews, each labeled with a genre, either comedy or action:

| review                      | class  |
|-----------------------------|--------|
| fun, couple, love, love     | comedy |
| fast, furious, shoot        | action |
| couple, fly, fast, fun, fun | comedy |
| furious, shoot, shoot, fun  | action |
| fly, fast, shoot, love      | action |

And a new document D: 

| review                     | class  |
|----------------------------|--------|
| fast, couple, shoot, fly   | ?      |

We have to compute the most likely class for D.

Let's start by saving our documents in some vectors:

In [ ]:
train_docs = [
    ['fun', 'couple', 'love', 'love'],
    ['fast', 'furious', 'shoot'],
    ['couple', 'fly', 'fast', 'fun', 'fun'],
    ['furious', 'shoot', 'shoot', 'fun'],    
    ['fly', 'fast', 'shoot', 'love']]

train_labels = ['comedy', 'action', 'comedy', 'action', 'action']
test_doc = ['fast', 'couple', 'shoot', 'fly']

### Computing the priors

Remember what we needed to do? The first step is to compute the **priors**. Let's do that with a simple function:

In [ ]:
# What are our classes?
classes = set(train_labels)
print(classes)

# initialise the priors
priors = {}
for _class in classes:
    priors[_class] = 0

# count how many train example in each class
for _class in classes:
    for label in train_labels:
        if _class == label:
            priors[_class] += 1

print(priors)

> **<h3>💻 Try it yourself!</h3>**

Now the priors are not *normalised*, i.e. we have to bring each prior in the range $[0,1]$. Can you do that in the following cell?

In [ ]:
# write your code here

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
    <p><pre><code>
for _class in classes:
    priors[_class] = priors[_class] / len(train_labels)
    </code></pre></p>

</details> 

If you've got the priors correct, you should have $P(comedy) = 0.4$ and $P(action)=0.6$.

In [ ]:
print(priors)

> **<h3>💻 Try it yourself!</h3>**

Now let's wrap everything nicely into a function. Can you complete the cell below?

In [ ]:
def compute_priors(labels):
    '''
    Computes the priors for a set of labels.
    '''

    # What are our classes?
    classes = set(labels)

    # -------------------------#
    #      E X E R C I S E     #
    # -------------------------#
    # initialise the priors
    priors = {}

    # ...?

    # count how many train example in each class
    
    # ...?
    
    # normalise the priors
    
    # ...?
    
    # ~      end exercise    ~ #
    
    return priors

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
  <p><pre><code>
def compute_priors(labels):
    '''
    Computes the priors for a set of labels.
    '''
    
    # What are our classes?
    classes = set(labels)

    # initialise the priors
    priors = {}
    for _class in classes:
        priors[_class] = 0

    # count how many train example in each class
    for _class in classes:
        for label in labels:
            if _class == label:
                priors[_class] += 1
    
    # normalise the priors
    for _class in classes:
        priors[_class] = priors[_class] / len(labels)
        
    return priors
  </code></pre></p>
</details> 

In [ ]:
priors = compute_priors(train_labels)
print(priors)

### Find the vocabulary

Since we need to compute the likelihoods for all words in our vocabulary, we need to find all the words in our corpus first. Let's build our vocabulary with a simple function:

In [ ]:
def create_vocabulary(lines):
    '''
    Creates a vocabulary from test.
    
    Parameters
    ----------
    lines: a list of lists of words
    
    Returns
    -------
    a set with all the words in the lines.
    
    '''
    
    words = set()
    for line in lines:
        for word in line:
            words.add(word)
            
    return words

In [ ]:
create_vocabulary(train_docs)

### Computing the likelihoods

Now we need to compute the likelihoods of the words w.r.t. to each class. To do that, we can build a `dict` for each class, where `dict[word] = P(word|class)`. To do that we can slightly modify the function `create_vocab_dict` that we defined in [Module 1.4](../../module_1/module_1.4/module_1.4.ipynb). The function we defined was the following:

In [ ]:
def create_vocab_dict(lines):
    '''
    Collect vocabulary counts from text

    Parameters
    ----------
    f_processed_arg: a list of lists of words

    Returns
    -------
    a dictionary with words (str) as keys and counts(int) as values
    vocab={
    'SONNETS': 1
    }
    '''
    vocab={}# create an empty vocabulary dictionary to store words as keys and counts as values later. 
    for line in lines:
        for word in line:
            if word in vocab:
                vocab[word]+=1 # update the count for an existing word
            else:
                vocab[word]=1 # initilize the count for a new word
    return vocab

In [ ]:
print(create_vocab_dict(train_docs))

> **<h3>💻 Try it yourself!</h3>**

How can we modify this function to give us the likelihoods for each class? Modify the function below, directly derived from `create_vocab_dict`, to return the likelihoods instead of the raw counts.

In [ ]:
def compute_likelihoods(lines, vocabulary):
    '''
    Computes the likelihoods of words in a list of strings.

    Parameters
    ----------
    lines: a list of list of words
    vocabulary: the vocabulary of the full corpus

    Returns
    -------
    a dictionary with words (str) as keys and likelihoods(floats) as values
    vocab={
    'SONNETS': 0.01
    }
    '''
    
    
    # -------------------------#
    #      E X E R C I S E     #
    # -------------------------#
    # create an empty vocabulary dictionary to store words 
    # as keys and counts as values later. 
    
    likelihoods = {}
    
    # initialise the likelihoods
    # hint: iterate through the vocabulary and initialise
    # a new element of the likelihoods dict to 0

    
    
    # ~      end exercise    ~ #    
    
    # Now we iterate through the lines
    for line in lines:
        for word in line:
            likelihoods[word] += 1 

    # -------------------------#
    #      E X E R C I S E     #
    # -------------------------#
    # how long are our documents?
    total_tokens = 0
    
    # write your code here
    # (hint: sum the length of the lines in total_tokens!)
    
    for word in likelihoods:
        likelihoods[word] = # Write your code here
    
    # ~      end exercise    ~ #

    return likelihoods

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
    <p><pre><code>
def compute_likelihoods(lines, vocabulary):
    '''
    Computes the likelihoods of words in a list of strings.

    Parameters
    ----------
    lines: a list of list of words
    vocabulary: the vocabulary of the full corpus

    Returns
    -------
    a dictionary with words (str) as keys and likelihoods(floats) as values
    vocab={
    'SONNETS': 0.01
    }
    '''
    
    
    # -------------------------#
    #      E X E R C I S E     #
    # -------------------------#
    # create an empty vocabulary dictionary to store words 
    # as keys and counts as values later. 
    
    likelihoods = {}
    
    # initialise the likelihoods
    # hint: iterate through the vocabulary and initialise
    # a new element of the likelihoods dict to 0
    for word in vocabulary:
        likelihoods[word] = 0
    
    
    # ~      end exercise    ~ #    
    
    # Now we iterate through the lines
    for line in lines:
        for word in line:
            likelihoods[word] +=1 

    # -------------------------#
    #      E X E R C I S E     #
    # -------------------------#
    # how long are our documents?
    total_tokens = 0
    
    # write your code here
    # (hint: sum the length of the lines in total_tokens!)
    for line in lines:
        for word in line:
            total_tokens +=1
    
    for word in likelihoods:
        likelihoods[word] = likelihoods[word]/total_tokens
    
    # ~      end exercise    ~ #

    return likelihoods
  </code></pre></p>
</details> 

In [ ]:
print(compute_likelihoods(train_docs, create_vocabulary(train_docs)))

Note that this method computes the likelihoods of the words in the whole vocabulary, irregardless of the classes. To get the likelihoods for each class we need to do as follows:

In [ ]:
target_class = 'action'
target_docs = []
vocabulary = create_vocabulary(train_docs)

# enumerate builds an (index, doc) list, hence allowing
# us to retrieve the label for each doc
for i, doc in enumerate(train_docs):
    if train_labels[i] == target_class:
        target_docs.append(doc)
        
print(target_docs)
print(compute_likelihoods(target_docs, vocabulary))

### The training function

> **<h3>💻 Try it yourself!</h3>**

Now we have all the instruments to build our training function. Can you complete it?

In [ ]:
def train_naive_bayes(documents, labels):
    
    classes = set(labels)
    
    # compute the priors
    priors = compute_priors(labels)
    vocabulary = create_vocabulary(documents)
    
    # this dict will contain the likelihoods, e.g.
    # likelihoods['action'] = {'fast': 0.2, 'furious': 0.1...
    likelihoods = {}
    
    # -------------------------#
    #      E X E R C I S E     #
    # -------------------------#
    for _class in classes:
        # get the documents belonging to each class:
        class_docs = []

        # put your code here
        for #...
        
        # compute the likelihood of this class
        likelihoods[_class] = # hint: use compute_likekihood defined above
        
    # ~      end exercise    ~ #
        
    return priors, likelihoods    

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
    <p><pre><code>
def train_naive_bayes(documents, labels):

    classes = set(labels)

    # compute the priors
    priors = compute_priors(labels)
    vocabulary = create_vocabulary(documents)

    # this dict will contain the likelihoods, e.g.
    # likelihoods['action'] = {'fast': 0.2, 'furious': 0.1...
    likelihoods = {}

    for _class in classes:
        # get the documents belonging to each class:
        class_docs = []

        for i, doc in enumerate(documents):
            if labels[i] == _class:
                class_docs.append(doc)

        # compute the likelihood of this class
        likelihoods[_class] = compute_likelihoods(class_docs, vocabulary)

    return priors, likelihoods 
    </code></pre></p>

</details> 

In [ ]:
priors, likelihoods = train_naive_bayes(train_docs, train_labels)

print('Priors:')
print(priors)

print('')
print('Likelihoods:')
print(likelihoods)

### Predict unknown classes

So now we have trained our model. How can we predict the likelihood of new sentences belonging to each class? Remember from above that

$$
\hat{c} = \text{argmax}_{c \in C} 
       \underbrace{P(d)}_\text{prior}
       \underbrace{\prod_{w \in d}{P(w \mid c)}}_\text{likelihood} 
$$

So, to predict the class of our new sentence `fast, couple, shoot, fly`, we need to:
- for each class `c`, we need to calculate `prob_c` by
    - multiplying the probability of each word `w` given that class `c`

Then, we'll just have to look at the maximum of our `prob_c`s. So, can we predict the class of `fast, fun, love, fly`? Let's write a function to do this.

In [ ]:
def bayes_predict(document, priors, likelihoods):
    '''
    Predicts the label for a document given the trained
    priors and likelihoods.
    
    Parameters
    ----------
    document: the document to analyse
    priors: the trained priors
    likelihoods: the trained likelihoods
    
    Return
    ------
    The probability for each class.
    '''
    
    classes_probabilities = {}

    # unpack the dictionary and iterate 
    # through the priors
    for label, prior in priors.items():
        
        # initialise the probability of a class to its prior
        prob_class = prior
        for word in document:
            if word in likelihoods[label]:
                # multiply the prior for the likelihood of each word
                prob_class = prob_class*likelihoods[label][word]
        classes_probabilities[label] = prob_class
        
    return classes_probabilities
    

Good! Let's try it with:

In [ ]:
document = ['fast', 'fun', 'love', 'fly']
bayes_predict(document, priors, likelihoods)

So for this document, `comedy` is the most likely class! 

Now let's try with the document for the assignment:

In [ ]:
print(test_doc)
bayes_predict(test_doc, priors, likelihoods)

Now the classes are both zero! How come? Well, we didn't apply any smoothing, so obviously at some point we are multiplying the likelihoods by zero, since $P(couple \ \mid action) = 0$ and $P(shoot \mid love) = 0$.

### Adding 1-smoothing

> **<h3>💻 Try it yourself!</h3>**

How can we modify `compute_likelihoods` to add 1-smoothing? Please update the function below.

Note that:
- We added a parameter (`smoothing`) to select the smoothing mode
- You will need to change how to normalise the likelihoods.

In [ ]:
def compute_likelihoods(lines, vocabulary, smoothing):
    '''
    Computes the likelihoods of words in a list of strings.

    Parameters
    ----------
    lines: a list of list of words
    vocabulary: the vocabulary of the training corpus
    smooething: the smoothing method to use

    Returns
    -------
    a dictionary with the probability for each class
    '''
    
    likelihoods = {}

    # populate the likelihoods
    for word in vocabulary:
        likelihoods[word] = 0
 

    # Now we iterate through the lines to count
    # the appearances of each word
    for line in lines:
        for word in line:
            likelihoods[word] +=1 

    # how long are our documents?
    total_tokens = 0
    for line in lines:
        for word in line:
            total_tokens +=1

    # Apply smoothing, if needed
    for word in likelihoods:
        if smoothing == 'none':
            
            for line in lines:
                for word in line:
                    total_tokens +=1

            for word in likelihoods:
                likelihoods[word] = likelihoods[word]/total_tokens
            
        elif smoothing == 'add1':
            # -------------------------#
            #      E X E R C I S E     #
            # -------------------------#
            # calculate the smoothing parameter for each word.
            smoothing_param = # ...?
            
            likelihoods[word] = # ...?
            # ~      end exercise    ~ #
        else:
            print('Unknown smoothing!')
            return

    return likelihoods

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
    <p><pre><code>
def compute_likelihoods(lines, vocabulary, smoothing):
    '''
    Computes the likelihoods of words in a list of strings.

    Parameters
    ----------
    lines: a list of list of words
    vocabulary: the vocabulary of the training corpus
    smooething: the smoothing method to use

    Returns
    -------
    a dictionary with the probability for each class
    '''
    
    likelihoods = {}

    # populate the likelihoods
    for word in vocabulary:
        likelihoods[word] = 0
 

    # Now we iterate through the lines to count
    # the appearances of each word
    for line in lines:
        for word in line:
            likelihoods[word] +=1 

    # how long are our documents?
    total_tokens = 0
    for line in lines:
        for word in line:
            total_tokens +=1

    # Apply smoothing, if needed
    for word in likelihoods:
        if smoothing == 'none':
            
            for line in lines:
                for word in line:
                    total_tokens +=1

            for word in likelihoods:
                likelihoods[word] = likelihoods[word]/total_tokens
            
        elif smoothing == 'add1':
            # -------------------------#
            #      E X E R C I S E     #
            # -------------------------#
            # calculate the smoothing parameter for each word.
            smoothing_param = total_tokens + len(vocabulary)
            
            likelihoods[word] = (likelihoods[word] + 1)/smoothing_param
            # ~      end exercise    ~ #
        else:
            print('Unknown smoothing!')
            return

    return likelihoods
    </code></pre></p>

</details> 

Now let's update `train_naive_bayes` to instruct it to use smoothing, and let's see if the results we obtain are correct:

In [ ]:
def train_naive_bayes(documents, labels, smoothing):

    classes = set(labels)

    # compute the priors
    priors = compute_priors(labels)
    vocabulary = create_vocabulary(documents)

    # this dict will contain the likelihoods, e.g.
    # likelihoods['action'] = {'fast': 0.2, 'furious': 0.1...
    likelihoods = {}

    for _class in classes:
        # get the documents belonging to each class:
        class_docs = []

        for i, doc in enumerate(documents):
            if labels[i] == _class:
                class_docs.append(doc)

        # compute the likelihood of this class
        likelihoods[_class] = compute_likelihoods(class_docs, vocabulary, smoothing)

    return priors, likelihoods 

In [ ]:
priors, likelihoods = train_naive_bayes(train_docs, train_labels, 'add1')

print('Priors:')
print(priors)

print('')
print('Likelihoods:')
print(likelihoods)

In [ ]:
bayes_predict(test_doc, priors, likelihoods)

Great! We successfully implemented Naive Bayes in Python. 

### Improving Naive Bayes: `argmax` and log-likelihoods

What we've done until now is good - but there are still a couple of open issues. 
- First, we should note that the original function of Naive Bayes is based on $\text{argmax}$: how do we implement it in Python?
- Then, we may note that the probabilities returned by our model are very small (i.e. in the order of $10^{-5}$), which means that for big vocabularies we may reach probabilities close to zero.

While the latter may not sound like a big issue for you, unfortunately dealing with very small numbers may be problematic for a computer - as mentioned in Module 1.2, computers use [floating-point arithmetic](https://en.wikipedia.org/wiki/Floating-point_arithmetic), which in practice means that they aren't actually very good at dealing with very small or very big numbers.

Actually, machines are not very good at representing any number that can't be expressed as a sum of powers of $2$. Take for example $0.1$. If we ask Python to show it, it will print

In [ ]:
0.1

Which looks fine. But what if we ask Python to show the first 40 decimal digits of $0.1$? That shouldn't make sense, right? Let's see:

In [ ]:
print(f'{0.1:.40f}')

See? Under the hood Python represents 0.1 as the infinitely repeating binary fraction `0.0001100110011001100110011001100110011001100110011...`. Since we stop at 32 bits, we won't be able to exactly represent $0.1$, but we will get an approximation, i.e. the value printed in the cell above.

This does not mean that our code is buggy or that Python is getting it wrong. It's just how computers work! Unfortunately, this means that, since Naive Bayes will generate very small numbers, it will be more prone to error. For this reason, we use the **log-likelihoods** instead of the actual likelihoods, i.e. we will rewrite:
$$
\begin{align}
\hat{c} &= \text{argmax}_{c \in C} 
       \underbrace{P(c)}_\text{prior}
       \underbrace{\prod_{w \in d}{P(w \mid c)}}_\text{likelihood} \\
       &= \text{argmax}_{c \in C} \log(  
       \underbrace{P(c)}_\text{prior}
       \underbrace{\prod_{w \in d}{P(w \mid c)}}_\text{likelihood} ) \\
       &= \text{argmax}_{c \in C} 
       \underbrace{\log P(c)}_\text{prior} +
       \underbrace{\sum_{w \in d}{\log P(w \mid c)}}_\text{likelihood} 
\end{align}
$$

To do that, we will use `numpy`, arguably Python's most popular mathematical library, which offers the function `log`.

In the next cells we'll install numpy (if it's not already installed), and the we'll import it. You'll almost always find numpy imported as `np` - so if in any Python code you see something like `np.` you can assume that it's based on numpy.

In [ ]:
!pip install numpy

In [ ]:
# import and test with ln(e), i.e. the logarithm to the base e of e
import numpy as np
np.log(np.e)

Numpy also offers a convenient `argmax` function. Remember that `argmax` works by selecting the *index* of the biggest element of a list:

In [ ]:
np.argmax([0,-2,10,3])   # Remember that we use 0-based indexing!

Now can wrap everything in an updated `bayes_predict_log` function, which will be the log-based version of `bayes_predict`. Can you update it yourself?

> **<h3>💻 Try it yourself!</h3>**

In [ ]:
def bayes_predict_log(document, priors, likelihoods):
    '''
    Predicts the label for a document given the trained
    priors and likelihoods.
    
    Parameters
    ----------
    document: the document to analyse
    priors: the trained priors
    likelihoods: the trained likelihoods
    
    Return
    ------
    A tuple (best_class, probabilities), where the first element
    is the name of the best class, and the second element is the dictionary of
    the computed probabilities.
    '''
    
    classes_probabilities = {}

    # unpack the dictionary and iterate 
    # through the priors
    for label, prior in priors.items():
        
        # -------------------------#
        #      E X E R C I S E     #
        # -------------------------#
        # initialise the probability of a class to the log of its its prior
        prob_class = # ...?
        for word in document:
            if word in likelihoods[label]:
                # sum the prior with the log-likelihood of each word
                prob_class =  # ...?
        classes_probabilities[label] = prob_class
    
    # get the names of the classes
    class_names = list(priors.keys())

    # -------------------------#
    #      E X E R C I S E     #
    # -------------------------#
    # complete the next line by selecting the name of the best class.
    # hint: the probabilities are saved in the dictionaries, and
    # you can access them by using classes_probabilities.values().
    # note that you can use more than 1 line if you need!
    
    best_class = # ...?

    # ~      end exercise    ~ #
    
    return best_class, classes_probabilities    

<hr>    <!-- please remember this! -->
<details>
  <summary>Click <b>here</b> to see the answer.</summary>
  <p><pre><code>
def bayes_predict_log(document, priors, likelihoods):
    '''
    Predicts the label for a document given the trained
    priors and likelihoods.
    
    Parameters
    ----------
    document: the document to analyse
    priors: the trained priors
    likelihoods: the trained likelihoods
    
    Return
    ------
    A tuple (best_class, probabilities), where the first element
    is the name of the best class, and the second element is the dictionary of
    the computed probabilities.
    '''
    
    classes_probabilities = {}

    # unpack the dictionary and iterate 
    # through the priors
    for label, prior in priors.items():
        
        # -------------------------#
        #      E X E R C I S E     #
        # -------------------------#
        # initialise the probability of a class to the log of its its prior
        prob_class = np.log(prior)
        for word in document:
            if word in likelihoods[label]:
                # sum the prior with the log-likelihood of each word
                prob_class = prob_class + np.log(likelihoods[label][word])
        classes_probabilities[label] = prob_class
    
    # get the names of the classes
    class_names = list(priors.keys())

    # -------------------------#
    #      E X E R C I S E     #
    # -------------------------#
    # complete the next line by selecting the name of the best class.
    # hint: the probabilities are saved in the dictionaries, and
    # you can access them by using classes_probabilities.values()
    best_class = class_names[np.argmax(classes_probabilities.values())]

    # ~      end exercise    ~ #
    
    return best_class, classes_probabilities      
  </code></pre></p>

</details> 

In [ ]:
document = ['fast', 'couple', 'shoot', 'fly']
bayes_predict_log(document, priors, likelihoods)

Good! Now the values are more reasonable and we can expect our computer to make less errors. Also, the code will run quicker, as summing typically requires less computational time than multiplying.

### Other improvements

Naturally, we could improve our Naive Bayes implementation even further. We could try different smoothing techniques, we could cache the log-likelihoods in the `likelihoods` dictionary instead of applying `np.log` every time, and so on. For now, let's just be happy with what we've done, and let's apply our code to the data of the previous lecture.

## Re-classify the *Thumbs up?* dataset

### Recap of loader functions

### Applying our model

### Differences between our model and `scikit-learn`

## Wrapping up

### Additional resources

- Naive Bayes chapter from [Manning, Raghavan and Schütze's Introduction to Information Retrieval](https://nlp.stanford.edu/IR-book/html/htmledition/naive-bayes-text-classification-1.html)
- Python's official documentation on [floating-point arithmetics and its issues](https://docs.python.org/3/tutorial/floatingpoint.html)
-  `scikit-learn`'s [implementation of Naive Bayes](https://github.com/scikit-learn/scikit-learn/blob/0fb307bf3/sklearn/naive_bayes.py#L669)
- Zhai and Lafferty, *A study of smoothing methods for language models applied to information retrieval*: a paper comparing different smoothing techniques for language models. [[pdf](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.58.8978&rep=rep1&type=pdf)]